In [1]:
import codecs
import gensim
from sklearn import  preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np
import xgboost as xgb
from tqdm import tqdm

##  1 数据准备

In [2]:
# !wget https://raw.githubusercontent.com/foochane/text-classification/master/output/data_clean_split.txt

In [3]:
# 读取数据
labels = []
text = []
with codecs.open('output/data_clean_split.txt','r',encoding='utf-8') as f:
    document_split = f.readlines()
    for document in document_split:
        temp = document.split('\t')
        labels.append(temp[0])
        text.append(temp[1].strip())  

# 标签转换为数字
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# 将每个句子切分成单个词
text_s2w= [s.split() for s in text]

## 2 构建word2vec模型

### 2.1 训练word2vec模型


In [4]:
model = gensim.models.Word2Vec(text_s2w,
                               min_count=5,
                               workers=6,
                               window =8,
                               size=200)

参数说明：

- min_count: 对于词频 < min_count 的单词，将舍弃（其实最合适的方法是用 UNK 符号代替，即所谓的『未登录词』，这里我们简化起见，认为此类低频词不重要，直接抛弃）

- workers: 可以并行执行的核心数，需要安装 Cython 才能起作用（安装 Cython 的方法很简单，直接 pip install cython）

size: 词向量的维度，神经网络隐层节点数

- window: 目标词汇的上下文单词距目标词的最长距离，很好理解，比如 CBOW 模型是用一个词的上下文预测这个词，那这个上下文总得有个限制，如果取得太多，距离目标词太远，有些词就没啥意义了，而如果取得太少，又信息不足，所以 window 就是上下文的一个最长距离

### 2.2 word2vec模型的简单使用
#### 2.2.1 构建词建词嵌入字典

In [5]:

embeddings_index = dict(zip(model.wv.index2word, model.wv.vectors))
print('Found %s word vectors.' % len(embeddings_index))

Found 87117 word vectors.


### 2.2.2 获取某个词的向量

In [6]:
model['汽车']

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.91600686,  0.65269667,  0.7890408 ,  0.2989293 , -0.6410602 ,
       -0.7721046 ,  0.17112012, -1.7168757 , -0.26790395, -0.813138  ,
        1.9748048 , -0.23864678, -1.7292303 ,  0.6713079 , -1.8142852 ,
       -0.85701007, -0.6777396 ,  0.9430919 , -0.3778572 , -1.0770824 ,
       -1.4120737 , -0.20698853, -1.0127343 ,  0.4006992 , -0.25051454,
       -0.7961283 ,  1.0942363 , -1.0193537 , -1.0111535 , -0.7164076 ,
        0.35792065,  0.71901995,  1.3802037 ,  0.6021403 ,  0.6371317 ,
       -0.10652476, -0.5859538 , -0.42821026,  1.1720324 ,  2.0588932 ,
        0.66107875,  0.71411633, -0.62187326,  0.06239692,  0.8000502 ,
        1.743869  , -1.4786941 , -1.025975  , -0.80224764, -0.8246677 ,
       -0.27273858,  0.38516802, -0.10491314,  0.00718853, -1.2467189 ,
       -2.088165  ,  1.0662181 , -1.9617743 , -0.70589083,  1.4909247 ,
        0.47113743,  0.4980447 ,  0.85836077,  0.06246256,  1.0078732 ,
        0.58501107, -0.7112168 ,  1.296002  , -0.7704843 ,  2.18

### 2.2.3 查看某个词的与其他词的相似度

In [7]:
model.most_similar('人民日报')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('光明日报', 0.8739441633224487),
 ('年月日', 0.8058885931968689),
 ('海外版', 0.8037006855010986),
 ('经济日报', 0.7827633619308472),
 ('中国青年报', 0.7719920873641968),
 ('社论', 0.7691082954406738),
 ('新民晚报', 0.761530876159668),
 ('解放日报', 0.7490832805633545),
 ('中国文联', 0.745826005935669),
 ('文汇报', 0.738906741142273)]

### 2.2.4 保存模型

In [8]:
model.save('/tmp/w2v_model')

### 2.2.5 加载模型

In [9]:
model_load = gensim.models.Word2Vec.load('/tmp/w2v_model')

## 3 训练数据处理



In [10]:
#该函数会将语句转化为一个标准化的向量（Normalized Vector）
def sent2vec(s):
    """
    将每个句子转换会一个100的向量
    """
    words = s.split()
    M = []
    for w in words:
        try:
            #M.append(embeddings_index[w])
            M.append(model[w])
        except:
            continue
    M = np.array(M)  # shape=(x,100),x是句子中词的个数，100是每个词向量的维数
    v = M.sum(axis=0) # 维度是100，对M中的x个数求和，得到每一维度的总和
    if type(v) != np.ndarray: 
        return np.zeros(200)
        print("np.zeros(200)")
    
    return v / np.sqrt((v ** 2).sum()) # 正则化，最后每个句子都变为一100维的向量

In [11]:
# 对训练集和验证集使用上述函数，进行文本向量化处理
text_s2v = [sent2vec(s) for s in tqdm(text)]

# 转换成numpy array数组
text_s2v = np.array(text_s2v)

# 切分数据集
from sklearn.model_selection import train_test_split
x_train_w2v, x_valid_w2v, y_train, y_valid = train_test_split(text_s2v, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

  0%|          | 0/9249 [00:00<?, ?it/s]/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 9249/9249 [01:05<00:00, 142.16it/s]


## 4 调用模型进行分类

In [12]:
# 定义损失函数
def multiclass_logloss(actual, predicted, eps=1e-15):
    """对数损失度量（Logarithmic Loss  Metric）的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [13]:
# 基于word2vec特征在一个简单的Xgboost模型上进行拟合
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(x_train_w2v, y_train)
predictions = clf.predict_proba(x_valid_w2v)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.390 



size = 100: logloss: 0.375 /0.368 